In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table,Column,join
from astropy.io import fits
from astropy.constants import c
from astropy.time import Time
import astropy.units as u
import yaml
import os
from glob import glob
from tqdm.notebook import tqdm

In [2]:
SD_DIR = '/Users/mbedell/python/stellardiff/'
DATA_DIR = '/Users/mbedell/Documents/Research/tauceti/tauceti_lily/'

In [3]:
import sys
sys.path.append(SD_DIR)

In [4]:
import stellardiff as sd
sd

<module 'stellardiff' from '/Users/mbedell/python/stellardiff/stellardiff/__init__.py'>

In [5]:
# load in CHIRON-accessible Fe lines only
transitions = sd.linelist.LineList.read(SD_DIR+"sun_linelist.moog")
mask = (transitions['wavelength'] > 4505.) & (transitions['species'] >= 26.0) & (transitions['species'] < 27.0)
transitions = transitions[mask]

In [6]:
# set up profile
with open(SD_DIR+"sun_settings.yaml", "r") as fp:
    profile_settings = yaml.load(fp, Loader=yaml.Loader)

In [7]:
# just copying the script here
transitions["equivalent_width"] = np.nan * np.ones(len(transitions))
transitions["equivalent_width_err_pos"] = np.nan * np.ones(len(transitions))
transitions["equivalent_width_err_neg"] = np.nan * np.ones(len(transitions))

quality_constraints = dict(
  abundance=[-10, 10],
  abundance_uncertainty=[0, 1],
  equivalent_width=[1, 1000],
  equivalent_width_percentage_uncertainty=[0, 25],
  equivalent_width_uncertainty=[0, 1000],
  reduced_equivalent_width=[-10, -3],
)

overwrite_kwds = dict()#profile="gaussian")

In [8]:
input_spectra_path = glob(DATA_DIR+'achi*.fits')
all_stars = []
dates = []
MAKE_FIGURES = False

In [9]:
for input_spectrum in tqdm(input_spectra_path):
    star = os.path.basename(input_spectrum).split(".fits")[0]
    all_stars.append(star)
    
    if True:
    
        # read the spectrum:
        # TODO: check whether I need to sort wavelengths
        # TODO: switch to a less stupid method of combining orders
        with fits.open(input_spectrum) as hdus:
            wave = np.copy(hdus[0].data[:,:,0]).ravel()
            flux = np.copy(hdus[0].data[:,:,2]).ravel()
            ivars = (np.copy(hdus[0].data[:,:,3])**-2).ravel()
            rv = np.copy(hdus[0].header['BARYCORR'])
            rv += 16600. # systemic velocity ~ -16.6 km/s (SIMBAD)
            doppler = np.sqrt((1 + rv/c.value)/(1 - rv/c.value))
            wave *= doppler # barycentric correction
            try:
                date = Time(hdus[0].header['DATEOBS'])
            except KeyError:
                date = Time(hdus[0].header['DATE-OBS'])
            dates.append(date.jd)

        spectrum = sd.spectrum.Spectrum1D(wave,flux,ivars)

        models = []
        indices = []

        for i, settings in enumerate(profile_settings):
            # Skip bad lines.
            if not settings["metadata"]["is_acceptable"]:
                continue

            # Create a transitions mask.
            tm = np.in1d(transitions["hash"], settings["transition_hashes"])
            if np.all(tm == False): # TODO: check whether this is ok
                continue

            kwds = settings["metadata"].copy()
            kwds.update(overwrite_kwds)
            kwds.pop("is_acceptable", None)

            model = sd.model_spectrum.ProfileFittingModel(transitions[tm], **kwds)

            try:
                result = model.fit(spectrum)

            except:
                continue

            # Only use the line if it meets quality constraints.
            if "fitted_result" not in model.metadata \
            or not model.meets_quality_constraints(quality_constraints):
                continue

            # Save the equivalent width.
            ew, ew_err_neg, ew_err_pos = model.equivalent_width

            transitions["equivalent_width"][tm] = ew.to(10**-3 * u.Angstrom).value
            transitions["equivalent_width_err_neg"][tm] = ew_err_neg.to(10**-3 * u.Angstrom).value
            transitions["equivalent_width_err_pos"][tm] = ew_err_pos.to(10**-3 * u.Angstrom).value

            models.append(model)
            indices.append(np.where(tm)[0][0])

            if MAKE_FIGURES:
                fig = model.plot(spectrum)
                basename = "{element}-{wavelength:.0f}-{star}.png".format(
                    star=star,
                    element=model.transitions["element"][0].replace(" ", "-"),
                    wavelength=model.transitions["wavelength"][0])
                fig.savefig(DATA_DIR+basename)

                plt.close("all")
        transitions.write(DATA_DIR+star+'.moog', format="moog")


  0%|          | 0/11 [00:00<?, ?it/s]

In [10]:
# create the q2 input files:
with open(DATA_DIR+'stars.csv', 'w') as f:
    f.write('id,teff,logg,feh,vt\n')
    for star in all_stars:
        f.write('{0},5342,4.51,-0.53,0.86\n'.format(star)) # parameters from Costa Silva+2020

In [ ]:
data = Table(transitions['wavelength','species','expot','loggf'])
data['expot'].name = 'ep'
data['loggf'].name = 'gf'
for star in all_stars:
    ews = transitions.read(DATA_DIR+star+'.moog')
    data[star] = ews['equivalent_width']
data.write(DATA_DIR+'lines.csv', overwrite=True)

In [ ]:
# create the metadata file:
with open(DATA_DIR+'metadata.csv', 'w') as f:
    f.write('id,date\n')
    for star,date in zip(all_stars,dates):
        f.write('{0},{1}\n'.format(star, date))